In [ ]:
#Description:
#   Looking at AGENCY LEVEL DATA
#   lookups-controller : Endpoints pertaining to NIBRS Victim Demographic data (STATE)
#
#Definitions:
#https://github.com/fbi-cde/crime-data-frontend/blob/7d790a0327996eccf4f34730ff5145763eec806e/content/terms.json
#
#   "term": "Originating Reference ID (ORI)"- "definition": "A law enforcement agency’s unique identification number."
#   
#
#Modification History:
#   DD-MMM-YYY  Author          Description
#   16-07-2019  Stacey Smith    INITIAL CREATION



In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import csv

from pprint import pprint

# Import DATA.GOV API key ......................REMEMBER TO PLUG IN YOUR OWN KEY NAME HERE
from api_keys import ses_dg_key

In [ ]:
path = "output_data/state_data.csv"
state_data = pd.read_csv(path, encoding="ISO-8859-1")

state_data.head()

In [ ]:
#This is the same series created before
states = (state_data.iloc[:,2]).astype(str)
states

In [ ]:

#Opened an empty list to initialize a dataframe
empty_list = []

agencies = pd.DataFrame(empty_list)

#Creating columns on list based on the returned data for one state
agencies["State"] = ""
agencies["Origin"] = ""
agencies["Agency Name"] = ""
agencies["Agency Type Name"] = ""
agencies["State Abbr"] = ""
agencies["Division Name"] = ""
agencies["Region Name"] = ""
agencies["Region Desc"] = ""
agencies["County Name"] = ""
agencies["NIBRS"] = ""
agencies["Latitude"] = ""          
agencies["Longitude"] = ""
agencies["NIBRS Start Date"] = ""

#This is the loop to fill the dataframe.  This should work for any of us pulling the data from the FBI
#state APIs, with just tweaking the url, keyword and return information.

#The loop stores the return results for an individual state.  Then it loops through the return results to 
#populate each row in the empty dataframe created before.  Once it has looped through all of the return info,
#it goes back to the beginning and does the same for the next state in states.

#I found the "dataframe.append" bit on a website and it turned out to be the key to filling in the dataframe
#because it ignores index numbers and just appends to the next empty row available.  

for i in states:
    
    state = i 
    
    agency_query_url = "https://api.usa.gov/crime/fbi/sapi/api/agencies/byStateAbbr/"+ state +"?api_key=" + ses_dg_key
    agency_response_state = requests.get(agency_query_url).json()
    
    for i in agency_response_state['results']:
        
        agencies = agencies.append({'State': state, 
                          'Origin': i['ori'], 
                          'Agency Name': i['agency_name'],
                          'Agency Type Name' : i['agency_type_name'],
                          'State Abbr' : i['state_abbr'],
                          'Division Name' : i['division_name'],
                          'Region Name' : i['region_name'],
                          'Region Desc' : i['region_desc'],
                          'County Name' : i['county_name'],
                          'NIBRS' : i['nibrs'],
                          'Latitude' : i['latitude'],
                          'Longitude' : i['longitude'],
                          'NIBRS Start Date' : i['nibrs_start_date']},
                          ignore_index = True)
    

agencies.to_csv("output_data/agencies.csv") 

In [7]:
agencies.head()

,State,Origin,Agency Name,Agency Type Name,State Abbr,Division Name,Region Name,Region Desc,County Name,NIBRS,Latitude,Longitude,NIBRS Start Date
0,AK,AK0010100,Anchorage Police Department,City,AK,Pacific,West,Region IV,ANCHORAGE,False,61.1743,-149.284,None
1,AK,AK0010200,Fairbanks Police Department,City,AK,Pacific,West,Region IV,FAIRBANKS NORTH STAR,False,64.8394,-147.719,None
2,AK,AK0010300,Juneau Police Department,City,AK,Pacific,West,Region IV,JUNEAU,False,58.3566,-134.507,None
3,AK,AK0010400,Ketchikan Police Department,City,AK,Pacific,West,Region IV,KETCHIKAN GATEWAY,False,55.4499,-131.107,None
4,AK,AK0010500,Kodiak Police Department,City,AK,Pacific,West,Region IV,KODIAK ISLAND,False,57.8049,-152.373,None


In [125]:
#Gives me a total count of all agencies, both True and False for NIBRS
NIBRS_data = agencies.groupby(['State']).count()


#Gives me a series that contains just the counts of True and False NIBRS by State
NIBRS_data_s = agencies.groupby(['State', 'NIBRS']).size()

#Put the series
NIBRS_df = pd.DataFrame(NIBRS_data_s)
NIBRS_df.reset_index(level=['State', 'NIBRS'], inplace=True)



all_agency = pd.merge(NIBRS_data, NIBRS_df, on=['State'])
del all_agency['Origin']
del all_agency['Agency Name']
del all_agency['Agency Type Name']
del all_agency['State Abbr']
del all_agency['Division Name']
del all_agency['Region Name']
del all_agency['Region Desc']
del all_agency['County Name']
del all_agency['Latitude']
del all_agency['Longitude']
del all_agency['NIBRS Start Date']


t_all_agency = all_agency.loc[all_agency['NIBRS_y']==True, :]

all_agencies = t_all_agency.rename(columns={
    "NIBRS_x": "Total Agencies",
    0: "Active Agencies"
    
})

del all_agencies['NIBRS_y']

all_agencies['Pct Active'] = (all_agencies['Active Agencies']/all_agencies['Total Agencies'])*100
all_agencies

,State,Total Agencies,Active Agencies,Pct Active
2,AL,423,1,0.236407
4,AR,310,294,94.838710
6,AZ,124,5,4.032258
9,CO,244,220,90.163934
11,CT,107,94,87.850467
13,DC,3,1,33.333333
14,DE,62,62,100.000000
19,HI,4,1,25.000000
21,IA,254,241,94.881890
23,ID,113,109,96.460177


In [89]:
all_agencies.to_csv("output_data/agency_data.csv")

array(['AL', 'AR', 'AZ', 'CO', 'CT', 'DC', 'DE', 'HI', 'IA', 'ID', 'IL',
       'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS',
       'MT', 'NE', 'ND', 'NH', 'NM', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
       'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV'], dtype=object)